In [16]:
import matplotlib.pyplot as plt

from models import TransformerModelLooped, TransformerModelLoopedFirstNTokens
from curriculum import CurriculumSimple
from train import train_without_config, validate_model

n_dims = 10

In [18]:
model_loop_b5_n_1 = TransformerModelLoopedFirstNTokens(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
    n=1
).cuda()

cirriculum_b5_n_1  = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [5000, 5, 0])

metrics_l1_b5_n_1  = train_without_config(
    model_loop_b5_n_1, cirriculum_b5_n_1,
    model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

number of parameters: 0.79M


  0%|          | 0/15000 [00:00<?, ?it/s]

0


loss 8.89684772491455: 100%|██████████| 15000/15000 [16:22<00:00, 15.27it/s]  


In [ ]:
model_loop_b5_n_30 = TransformerModelLoopedLastNTokens(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
    n=5
).cuda()

cirriculum_b5_n_30 = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [5000, 5, 0])

metrics_l1_b5_n_30 = train_without_config(
    model_loop_b5_n_30, cirriculum_b5_n_30,
    model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

number of parameters: 0.79M


loss 9.596034049987793:   0%|          | 2/15000 [00:00<18:30, 13.50it/s]

0


loss 10.277716636657715:  66%|██████▌   | 9904/15000 [10:35<05:37, 15.10it/s]

In [4]:
model_loop_b5_n_31 = TransformerModelLoopedLastNTokens(
    n_dims=n_dims,
    n_positions=101,
    n_embd=256,
    n_layer=1,
    n_head=4,
    pred_type="regression",
    n=31
).cuda()

cirriculum_b5_n_31 = CurriculumSimple(n_dims, 31, 5, [5000, n_dims, 0], [5000, 31, 0], [5000, 5, 0])

metrics_l1_b5_n_31 = train_without_config(
    model_loop_b5_n_31, cirriculum_b5_n_31, 
    model_n_dims=n_dims, log_every_steps=10, train_steps=15000, family="gpt2_loop", do_wandb_log=False)

In [12]:
def calculate_by_loop(model):
    val_by_loop = []
    loop_steps = []
    for i in range(1, 60):
        val_loss = validate_model(model, n_dims_truncated=10, n_loops=i, model_n_dims=10, n_points=31, family="gpt2_loop")
        val_by_loop.append(val_loss / n_dims)
        loop_steps.append(i)
    return loop_steps, val_by_loop 
steps_b5_n_10, values_b5_n_10 = calculate_by_loop(model_loop_b5_n_10)
steps_b5_n_30, values_b5_n_30 = calculate_by_loop(model_loop_b5_n_30)
steps_b5_n_31, values_b5_n_31 = calculate_by_loop(model_loop_b5_n_31)

In [ ]:
def calculate_by_n_points(model, n_loops=5):
    vals = []
    loop_steps = []
    for i in range(1, 60):
        val_loss = validate_model(model, n_dims_truncated=10, n_loops=n_loops, model_n_dims=10, n_points=i, family="gpt2_loop")
        vals.append(val_loss / n_dims)
        loop_steps.append(i)
    return loop_steps, vals

steps_points_b5_n_10, values_points_b5_n_10 = calculate_by_n_points(model_loop_b5_n_10)
steps_points_b5_n_30, values_points_b5_n_30 = calculate_by_n_points(model_loop_b5_n_30)
steps_points_b5_n_31, values_points_b5_n_31 = calculate_by_n_points(model_loop_b5_n_31)

In [14]:
plt.plot(steps_b5_n_10, values_b5_n_10)
plt.plot(steps_b5_n_30, values_b5_n_30)
plt.plot(steps_b5_n_31, values_b5_n_31)
plt.legend(["n=10", "n=30", "n=31"])
plt.savefig('../images/check_for_convergence_properties.png')